In [30]:
import urllib.parse
from rdflib import Graph, Literal, Namespace, URIRef, BNode
from rdflib.namespace import RDF, RDFS, FOAF, OWL, XSD, DC, DCTERMS 
import pandas as pd

In [31]:
base_uri = "http://www.sanitasicilia.it/resource/"

g = Graph()
count = 0

sso = Namespace("http://www.sanitasicilia.it/ontology/")

g.bind("sso", sso)

ssr = Namespace("http://www.sanitasicilia.it/resource/")
g.bind("ssr", ssr)


In [32]:
def urify(ns, testo):
    testo=testo.replace(" ","_").replace("\'","")
    return ns+urllib.parse.quote(testo)

In [33]:
count = 0
row_prev = pd.Series()

def addBNodes(res, row):
    bn = BNode()

    g.add([res, sso.hasSite, bn])
    g.add([bn, sso.hasAddress, Literal(row[0], datatype=XSD.string) ])
   
    if(row[3] != 0):
        g.add([bn, sso.hasCap, Literal(row[3], datatype=XSD.integer) ])
    g.add([bn, sso.isIn, URIRef(urify(base_uri, row[4].title()))])
    g.add([bn, sso.tipology, Literal(row[7], datatype=XSD.string)])
    
    if(row[8] != 0 and row[9] != 0):
        g.add([bn, sso.hasLatitude, Literal(row[8], datatype=XSD.decimal) ])
        g.add([bn, sso.hasLongitude, Literal(row[9], datatype=XSD.decimal) ])

def addTriples(row):
    global count
    code = 'F'+f'{count:04}'
    res = URIRef(urify(base_uri, code))
    g.add([res, URIRef(RDF.type), URIRef(sso.Farmacia)])

    g.add([res, sso.hasName, Literal(row[1], datatype=XSD.string) ])
    g.add([res, sso.hasVatNumber, Literal(row[2], datatype=XSD.integer) ])

    addBNodes(URIRef(urify(base_uri, code)), row)
    count = count + 1


farmacie_df = pd.read_csv("../datasets/csv/completed/farmacie.csv")

farmacie_df.apply(lambda row : addTriples(row), axis=1)

g.serialize(destination="farmacie.ttl", format="turtle")

<Graph identifier=N50f421ac4d924ff9b6008fe82e39614c (<class 'rdflib.graph.Graph'>)>